### 추가 참고 : Automated Machine Learning with Auto-Keras @ http://j.mp/2xeG2c2

#### A. Keras library 를 활용한 방법

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, models, layers, utils, losses # tf.keras 에 필요한 함수들이 모여있습니다.

/Users/chaeyoon/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/chaeyoon/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/chaeyoon/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/chaeyoon/opt/anaconda3/lib/python3.7/site-packages/ten

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # https://stackoverflow.com/questions/35911252/disable-tensorflow-debugging-information

<br> 

#### 1. Prepare train & test data (MNIST)

#### Train & Test split

In [3]:
(train_data, train_label), (test_data, test_label) = datasets.mnist.load_data()

print(train_data.shape) # # of training data == 60000, each data = 28px * 28px
print(test_data.shape) # # of test data == 10000
print(train_data.dtype) # int
print(train_data.max()) # 0~255

(60000, 28, 28)
(10000, 28, 28)
uint8
255


#### Normalization

In [4]:
train_data = train_data.reshape(60000, 784).astype('float32') / 255.0 # 0~255 -> 0~1
test_data = test_data.reshape(10000, 784).astype('float32') / 255.0 # 0~255 -> 0~1

#### One-hot encoding

In [5]:
train_label

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [6]:
train_label = utils.to_categorical(train_label) # 0~9 -> one-hot vector
test_label = utils.to_categorical(test_label) # 0~9 -> one-hot vector

In [7]:
train_label

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

<br> 

#### 2. Build the model & Set the criterion 

**tensorflow layer 쌓는 방법**
1. Sequential
2. Functional

In [8]:
model = models.Sequential() # Build up the "Sequence" of layers (Linear stack of layers)

In [9]:
# Dense-layer (relu & he)
model.add(layers.Dense(input_dim=28*28, units=512, activation='relu', kernel_initializer='he_uniform')) 
model.add(layers.Dropout(0.2)) # Dropout-layer
# (Output) Dense-layer with softmax function, 0~9 -> 10
model.add(layers.Dense(units=10, activation='softmax')) 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
# "Compile" the model description (Configures the model for training)
model.compile(optimizer='adam', 
              loss=losses.categorical_crossentropy, # See other available losses @ https://keras.io/losses/
              metrics=['accuracy'])

<br> 

#### 3. Train the model

In [11]:
# Fit the model on training data

model.fit(train_data, train_label, batch_size=100, epochs=10) # default batch_size = 32

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
60000/60000 [==============================] - 13s 213us/sample - loss: 0.2725 - acc: 0.9219
Epoch 2/10
60000/60000 [==============================] - 11s 192us/sample - loss: 0.1152 - acc: 0.9663
Epoch 3/10
60000/60000 [==============================] - 12s 198us/sample - loss: 0.0795 - acc: 0.9760
Epoch 4/10
60000/60000 [==============================] - 11s 184us/sample - loss: 0.0588 - acc: 0.9821
Epoch 5/10
60000/60000 [==============================] - 12s 195us/sample - loss: 0.0471 - acc: 0.9856
Epoch 6/10
60000/60000 [==============================] - 11s 190us/sample - loss: 0.0383 - acc: 0.9884
Epoch 7/10
60000/60000 [==============================] - 12s 199us/sample - loss: 0.0310 - acc: 0.9905
Epoch 8/10
60000/60000 [==============================] - 12s 199us/sample - loss: 0.0266 - acc: 0.9917
Epoch 9/10
60000/60000 [==============================] - 12s 197us/sample - loss: 0.0238 - acc: 0.9922
Epoch 10/10
6000

<br> 

#### 4. Test the model

In [12]:
# Evaluate the model on test data

result = model.evaluate(test_data, test_label, batch_size=100)

10000/10000 [==============================] - 1s 81us/sample - loss: 0.0624 - acc: 0.9823


In [13]:
print('loss (cross-entropy) :', result[0])
print('test accuracy :', result[1])

loss (cross-entropy) : 0.0624014824797996
test accuracy : 0.9823


<br>

#### B. Tensorflow 의 keras API 를 활용한 방법 (@ https://goo.gl/v8h2SH)

In [14]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
# or
# x_train = tf.keras.utils.normalize(x_train, axis=1)  # scales data between 0 and 1
# x_test = tf.keras.utils.normalize(x_test, axis=1)  # scales data between 0 and 1

model = tf.keras.models.Sequential()  # a basic feed-forward model
model.add(tf.keras.layers.Flatten())  # takes our 28x28 and makes it 1x784
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu))  # a simple fully-connected layer, 512 units, relu activation
model.add(tf.keras.layers.Dropout(0.2))  
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))  # our output layer. 10 units for 10 classes. Softmax for probability distribution
# or
# model = tf.keras.models.Sequential([
#   tf.keras.layers.Flatten(),
#   tf.keras.layers.Dense(512, activation=tf.nn.relu),
#   tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.Dense(10, activation=tf.nn.softmax)
# ])

# if labels are not one-hot vectors, we should use "sparse_categorical_crossentropy" (http://j.mp/2XS0jmv)
model.compile(optimizer='adam',  # Good default optimizer to start with
              loss=losses.sparse_categorical_crossentropy,  # how will we calculate our "error." Neural network aims to minimize loss.
              metrics=['accuracy'])  # what to track

model.fit(x_train, y_train, epochs=5)

val_loss, val_acc = model.evaluate(x_test, y_test)  # evaluate the out of sample data with model
print('loss (cross-entropy) :', val_loss)
print('test accuracy :', val_acc)

# import numpy as np
# import matplotlib.pyplot as plt
# predictions = model.predict(x_test)
# print(np.argmax(predictions[0]))
# plt.imshow(x_test[0],cmap=plt.cm.binary)
# plt.show()

Epoch 1/5
60000/60000 [==============================] - 30s 495us/sample - loss: 0.2214 - acc: 0.9352
Epoch 2/5
60000/60000 [==============================] - 24s 407us/sample - loss: 0.0991 - acc: 0.9693
Epoch 3/5
60000/60000 [==============================] - 20s 327us/sample - loss: 0.0677 - acc: 0.9789
Epoch 4/5
60000/60000 [==============================] - 22s 365us/sample - loss: 0.0525 - acc: 0.9831
Epoch 5/5
10000/10000 [==============================] - 1s 115us/sample - loss: 0.0642 - acc: 0.9827
loss (cross-entropy) : 0.06416065907652374
test accuracy : 0.9827


<br>

#### C. AutoKeras 를 활용한 방법 (@ https://goo.gl/v8h2SH)

**\* PyTorch 및 AutoKeras 설치 & AutoKeras fitting 과정에 시간이 무척 많이 소요됩니다.**
<br>**\* 개발환경이 영향을 받을 수 있으니 Google Colab 에서 아래 코드를 실행해보는 것을 적극 권장합니다.**
<br>**\* 금일 실습을 마치고 추후 복습하실 때 살펴보세요.**
<br><br>

1) 먼저 OS에 맞는 pytorch를 설치해주어야 합니다. (아래 2가지 방식 중 택일, available pip whl files @ https://goo.gl/uYVaPa)

1. conda install pytorch=0.4.1 -c pytorch (추천)
2. pip install https://download.pytorch.org/whl/cpu/torch-0.4.1-cp36-cp36m-win_amd64.whl 

2) 그 다음 아래 명령어로 AutoKeras를 설치해줍니다.

- pip install autokeras

In [17]:
from keras import datasets, models, layers, utils
(train_data, train_label), (test_data, test_label) = datasets.mnist.load_data()

ModuleNotFoundError: No module named 'Keras'

In [ ]:
import autokeras as ak

model = ak.ImageClassifier() # It searches CNN architectures for the best configuration for the image dataset.
model.fit(train_data, train_label, time_limit=3600) # time_limit: The time limit for the search in seconds.

accuracy = model.evaluate(test_data, test_label)
result = model.predict(test_data)

print(accuracy)
print(result)

<br>

#### Appendix 
- Keras vs PyTorch @ https://goo.gl/ar5VPB